In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score

# ---- Load dataset ----
df = pd.read_json("D:\\Downloads\\archive\\News_Category_Dataset_v3.json", lines=True)
df = df[['headline','category']]

# Only 5 categories
categories = ["POLITICS", "SPORTS", "BUSINESS", "TECH", "CRIME"]
df = df[df['category'].isin(categories)]

# Encode categories as numbers
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(df['category'])

X = df['headline']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# TF-IDF
vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# ---- Train Linear Regression ----
model = LinearRegression()
model.fit(X_train_tfidf, y_train)

# Predict (regression outputs floats → round to nearest class)
y_pred = np.round(model.predict(X_test_tfidf)).astype(int)

# Clip values to valid category range
y_pred = np.clip(y_pred, 0, len(le.classes_)-1)

# Accuracy
print("Accuracy:", accuracy_score(y_test, y_pred))

# ---- Test user input ----
while True:
    user_input = input("Enter a news article (or 'quit'): ")
    if user_input.lower() == "quit":
        break
    
    user_tfidf = vectorizer.transform([user_input])
    pred = int(np.round(model.predict(user_tfidf))[0])
    pred = np.clip(pred, 0, len(le.classes_)-1)
    print("Predicted Category:", le.classes_[pred])


Accuracy: 0.7061520825372564
Predicted Category: POLITICS
